# Glacier surging experiments

Goals of this notebook:

- The student will be able to describe the effects of glacier surging.
- The student will be able to implement glacier surging in OGGM.

In this notebook we are going to explore the basic functionalities of OGGM flowline model(s) and perform a simple surging experiment. For this purpose we are going to use simple and "idealized" glacier models with simple linear mass-balance profiles. 

**What is glacier surge?**

A small percentage of glaciers undergo short periods of faster flow. They experience a change in morphology and surface characteristics, which sometimes leads to a marked frontal advance. The speed of the ice increases up to 10 - 1000 times of the normal velocity. Record flows are observed with velocities that exceed 100m per day. Surges happen in decadal cycles and can last for 1 up to 15 years.[¹](#Reference)

We start with importing all needed modules:

In [ ]:
# Plotting libraries and plot style
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('notebook')
sns.set_style('ticks')

# Scientific packages
import numpy as np

# Constants
from oggm import cfg
cfg.initialize_minimal()

# Mass-balance model
from oggm.core.massbalance import LinearMassBalance
# There are several numerical implementations in OGGM core. We use the "FluxBasedModel"
from oggm.core.flowline import FluxBasedModel as FlowlineModel
# Glacier shape
from oggm.core.flowline import RectangularBedFlowline

## Basics

We set up a simple glacier with a linear bedrock (see [getting started with flowline models](flowline_model.ipynb)) as a setting for our experiment.

In [ ]:
# This is the bed rock, linearily decreasing from 3400 m
# altitude to 1400 m, in 200 steps
nx = 200
bed_h = np.linspace(3400, 1400, nx)

# At the beginning, there is no glacier so our glacier surface is
# at the bed altitude
surface_h = bed_h

# Let's set the model grid spacing to 100 m 
map_dx = 100

# Calculate the corresponding distance along the glacier (from the top)
distance_along_glacier = np.linspace(0, nx, nx) * 0.1  # in km

# The units of widths is in "grid points", i.e. 3 grid points = 300 m
# in our case
widths = np.zeros(nx) + 3.

# Define our bed
initial_flowline = RectangularBedFlowline(surface_h=surface_h, bed_h=bed_h,
                                          widths=widths, map_dx=map_dx)

Then we define the mass balance model, we set the ELA to 3000 m a.s.l. and the gradient to 4 mm/m:

In [ ]:
# ELA at 3000m a.s.l., gradient 4 mm m-1
mb_model = LinearMassBalance(3000, grad=4)

Here we calculate a glacier that flows only because of deformation and  without sliding:

In [ ]:
# Define the year until we want to simulate the glacier.
year = 1200

# Initialize the model.
model = FlowlineModel(initial_flowline, mb_model=mb_model, y0=0.)

In [ ]:
# Simulate the glacier until
model.run_until(year)

In [ ]:
# Store the outline of the glacier as simple_glacier_h
simple_glacier_h = model.fls[-1].surface_h

In [ ]:
# Let's plot it!
plt.plot(distance_along_glacier, simple_glacier_h, label='Deforming glacier')
plt.plot(distance_along_glacier, bed_h, label='Bedrock', ls=':', c='k')
plt.xlabel('Distance along glacier [km]')
plt.ylabel('Altitude [m]')
plt.legend();

## Surging glacier experiment

Since surging occurs because of increased basal sliding, a surging period can be represented in the model by increasing the sliding parameter[¹](#Reference), for example by a factor of 10. Typically surging can be assumend to happen cyclically after 100 years of normal sliding and lasts over a period of 10 years.

Let's model a simple glacier that only slides:

In [ ]:
# Define parameters:
# Default in OGGM: Glen's creeping parameter
glen_a = cfg.PARAMS['glen_a']

# Sliding glaciers (sliding parameter fs is nonzero)
fs = 5.7e-20

# Define the year until we want to simulate the glacier.
year = 1200

Initialise the model. **Note that we pass Glens creeping parameter (`glen_a`) and the sliding parameter (`fs`) this time**

In [ ]:
# Initialize the model
model = FlowlineModel(initial_flowline, mb_model=mb_model, y0=0.,
                      glen_a=glen_a, fs=fs)

In [ ]:
# Simulate the glacier until 
model.run_until(year)

In [ ]:
# Store the final results for later use
sliding_glacier_h = model.fls[-1].surface_h

We can plot both the glaciers to see if there is any difference

In [ ]:
# Let's plot both models.
# The simple glacier
plt.plot(distance_along_glacier, simple_glacier_h, label='Deforming glacier')
# The sliding glacier
plt.plot(distance_along_glacier, sliding_glacier_h, label='Sliding glacier')
# The bed
plt.plot(distance_along_glacier, bed_h, label='Bedrock', ls=':', c='k')
plt.xlabel('Distance along glacier [km]')
plt.ylabel('Altitude [m]')
plt.legend();

Now let us simulate a surging glacier

In [ ]:
# Define parameters:
# Sliding parameter in times of slow sliding:
fs = 5.7e-20

# Sliding parameter in surging periods:
fs_surge = 5.7e-20*10

# Surging period (years):
period_s = 10

# Time span between 2 surging periods (years):
t_slow = 100

# Number of surges
no_surges = 10

For the surging experiment we will have to divide the simulation into different parts. The first step is to generate an array with the years for which we want to bin the simulation. During year with no surging we go in steps of ten while during the surges we take steps of one year.

In [ ]:
# 10 periods of 10 years surging every 100 years
yrs = np.arange(0, t_slow + 1, 10)
for i in range(no_surges):
    # Generate 10 one year increments from the last year.
    yrs_sliding = np.arange(t_slow+1+t_slow*i+period_s*i,
                            t_slow+period_s+1+t_slow*i+period_s*i,1)
    # Append them to the no surgning years
    yrs = np.append(yrs, yrs_sliding)

    # Generate another 100 years in 10 year increments.
    yrs_normal = np.arange(t_slow+period_s+10+t_slow*i+period_s*i,
                           2*t_slow+period_s+10+t_slow*i+period_s*i, 10)
    # Append it to the list.
    yrs = np.append(yrs, yrs_normal)

If it is still unclear to you which years are "surging years", take look at the array `yrs`

In [ ]:
yrs

Now it is time to set up the surging experiment. Similar to the experiments with the [mass balance gradients](mass_balance_gradients.ipynb), we cant simply change the parameters in the middle of the simulation. Instead we have to re-initialise a new model every time we want change any parameters. In this case it means running model **A** for 100 years, then running model **B** for 10 year, then back to model **A** for 100 year and so on.

In [ ]:
# Surging glaciers - model initialisation
model = FlowlineModel(initial_flowline, mb_model=mb_model, y0=0.,
                      glen_a=glen_a, fs=fs)

In [ ]:
# The number of steps
nsteps = len(yrs)
# Lists for saving indermidate data
length_surge_weak = []
volume_surge_weak = []
surging_glacier_h_weak = []

# We loop for the glacier evolution.
for i, yr in enumerate(yrs):
    # Simulate the glacier until the desired year.
    model.run_until(yr)
    # Save the states
    length_surge_weak.append(model.length_m)
    volume_surge_weak.append(model.volume_km3)
    
    # If we are at the first or last year, we do noting
    if i == 0 or i == (nsteps-1):
        continue
    
    # Check if the next year is a surging year
    elif (yr-yrs[i-1]) == 10 and (yrs[i+1]-yr) == 1:
        # Save the surface height before surge.
        surging_glacier_h_weak.append(model.fls[0].surface_h)
        # If it is, initialise a "new glacier" based on the 
        # old shape, but with a new sliding parameter.
        model = FlowlineModel(model.fls[0], mb_model=mb_model,
                              y0=yr, glen_a=glen_a, fs=fs_surge)

    # Check if the next year is not a surging year
    elif (yr-yrs[i-1]) == 1 and (yrs[i+1]-yr) == 10:
        # Save the surface height before surge.
        surging_glacier_h_weak.append(model.fls[0].surface_h)
        # If it is, initialise a "new glacier" based on the 
        # old shape, but with a new sliding parameter.
        model = FlowlineModel(model.fls[0], mb_model=mb_model,
                              y0=yr, glen_a=glen_a, fs=fs)

In the next figures the development of the glacier (length and volume) over the years is shown:


In [ ]:
# Plot it in two subplots
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

fig, axs = plt.subplots(2, 1, sharex=True, figsize=(14, 8))
# Spacing
fig.subplots_adjust(hspace=0)
# Specify the xticks.
plt.xticks(np.arange(min(yrs), max(yrs)+1, 100.0))
# Plot the glacier length
axs[0].plot(yrs, length_surge_weak, color='tab:green')
# Mark surging periods with orange patches.
for i, yr in enumerate(yrs):
    if (yr-yrs[i-1]) ==  1 and (yrs[i-1+period_s]-yrs[i+period_s-2]) == 1:
        axs[0].axvspan(yrs[i-1], yrs[i-1+period_s], color='tab:orange',
                       alpha=0.3)
# Label and grid.
plt.xlabel('Years')
axs[0].set_ylabel('Length [m]')
axs[0].grid()

# Plot the glacier volume
axs[1].plot(yrs, volume_surge_weak, color='tab:green')
# Mark surging periods with orange patches.
for i, yr in enumerate(yrs):
    if (yr-yrs[i-1]) ==  1 and (yrs[i-1+period_s]-yrs[i+period_s-2]) == 1:
        axs[1].axvspan(yrs[i-1], yrs[i-1+period_s], color='tab:orange',
                       alpha=0.3)
# Ylabel and grid.
axs[1].set_ylabel('Volume [km³]')
axs[1].grid()
# Define handles for the legend
label = [Line2D(yrs, volume_surge_weak, color='tab:green',
                label='Weakly surging glacier'),
         Patch(facecolor='tab:orange', alpha=0.3,
               edgecolor='r', label='Surging period')
        ]
# Add the legend
axs[0].legend(handles=label);

Now we compare the different calculated glacier:

In [ ]:
# Plot the results
f, ax = plt.subplots(figsize=(9, 6))
plt.plot(distance_along_glacier, simple_glacier_h, label='Deforming glacier')
plt.plot(distance_along_glacier, sliding_glacier_h, label='Sliding glacier')
# Plot the last surface height of the surging glacier.
plt.plot(distance_along_glacier, surging_glacier_h_weak[-1],
         label='Weakly surging glacier')

# Add the bedrock
plt.plot(distance_along_glacier, bed_h, label='Bedrock', ls=':', c='k')
plt.xlabel('Distance along glacier [km]')
plt.ylabel('Altitude [m]')
plt.legend();

We can increase the sliding parameter in order to increase the magnitude of the  surges:

In [ ]:
# Define parameters:
# Sliding parameter in times of normal sliding:
fs = 5.7e-20
# Sliding parameter in surging periods:
fs_surge = 5.7e-20*50

# Initialise the glacier.
model = FlowlineModel(initial_flowline, mb_model=mb_model, y0=0.,
                      glen_a=glen_a, fs=fs)

In [ ]:
# The number of steps
nsteps = len(yrs)
# Lists for saving indermidate data
length_surge_strong = []
volume_surge_strong = []
surging_glacier_h_strong = []

# We loop for the glacier evolution.
for i, yr in enumerate(yrs):
    # Simulate the glacier until the desired year.
    model.run_until(yr)
    # Save the states
    length_surge_strong.append(model.length_m)
    volume_surge_strong.append(model.volume_km3)
    
    # If we are at the first or last year, we do noting
    if i == 0 or i == (nsteps-1):
        continue
    
    # Check if the next year is a surging year
    elif (yr-yrs[i-1]) == 10 and (yrs[i+1]-yr) == 1:
        # Save the surface height before surge.
        surging_glacier_h_strong.append(model.fls[0].surface_h)
        # If it is, initialise a "new glacier" based on the 
        # old shape, but with a new sliding parameter.
        model = FlowlineModel(model.fls[0], mb_model=mb_model,
                              y0=yr, glen_a=glen_a, fs=fs_surge)

    # Check if the next year is not a surging year
    elif (yr-yrs[i-1]) == 1 and (yrs[i+1]-yr) == 10:
        # Save the surface height before surge.
        surging_glacier_h_strong.append(model.fls[0].surface_h)
        # If it is, initialise a "new glacier" based on the 
        # old shape, but with a new sliding parameter.
        model = FlowlineModel(model.fls[0], mb_model=mb_model,
                              y0=yr, glen_a=glen_a, fs=fs)

In [ ]:
# Plot the results
f, ax = plt.subplots(figsize=(9, 6))
# The simple and sliding glacier.
plt.plot(distance_along_glacier, simple_glacier_h, label='Deforming glacier')
plt.plot(distance_along_glacier, sliding_glacier_h, label='Sliding glacier')
# The weakly surgning glacier
plt.plot(distance_along_glacier, surging_glacier_h_weak[-1],
         label='Weak surging glacier')
# The more more strongly surging glacier.
plt.plot(distance_along_glacier, surging_glacier_h_strong[-1],
         label='Strong surging glacier')
# Add the bedrock
plt.plot(distance_along_glacier, bed_h, label='Bedrock', ls=':', c='k')
plt.xlabel('Distance along glacier [km]')
plt.ylabel('Altitude [m]')
plt.legend();

<div class="alert alert-success">
            <b>Compare the different glacier types. Can you explain the differences? </b>
</div>

In the next plot the development of the weakly surging glacier and the strongly surging glacier are shown:

In [ ]:
# Plot it in two subplots
fig, axs = plt.subplots(2, 1, sharex=True, figsize=(14, 8))
# Spacing
fig.subplots_adjust(hspace=0)
# xlabel
plt.xlabel('Years')
# Glacier length
axs[0].plot(yrs, length_surge_strong, color='tab:red')
axs[0].plot(yrs, length_surge_weak, color='tab:green')
# Mark surging periods
for i, yr in enumerate(yrs):
    if (yr-yrs[i-1]) == 1 and (yrs[i-1+period_s]-yrs[i+period_s-2]) == 1:
        axs[0].axvspan(yrs[i-1], yrs[i-1+period_s], color='tab:orange',
                       alpha=0.3)
# ylabel and grid.
axs[0].set_ylabel('Length [m]')
axs[0].grid()

# Glacier volume
axs[1].plot(yrs, volume_surge_strong, color='tab:red')
axs[1].plot(yrs, volume_surge_weak, color='tab:green')
# Mark surging periods
for i, yr in enumerate(yrs):
    if (yr-yrs[i-1]) ==  1 and (yrs[i-1+period_s]-yrs[i+period_s-2]) == 1:
        axs[1].axvspan(yrs[i-1], yrs[i-1+period_s], color='tab:orange',
                       alpha=0.3)
# ylabel and grid
axs[1].set_ylabel('Volume [km³]')
axs[1].grid()
# Define handles for legend.
label = [Line2D(yrs, length_surge_strong, color='tab:red',
                label='Strongly surging glacier'), 
         Line2D(yrs, length_surge_weak, color='tab:green',
                label='Weakly surging glacier'),
         Patch(facecolor='tab:orange', alpha=0.3, edgecolor='r',
               label='Surging period')
        ]
# Add legend.
axs[1].legend(handles=label);

<div class="alert alert-success">
    <details>
        <summary>
            <b>The response of the volume to a surge seems to be lagging behind the response in length somewhat. Can you come up with an explanation for this?  </b> <i>Click for a hint</i>
        </summary>
        At the first stage of a surge, the glacier is very quick to adjust its length. This doesn't really change the volume of the glacier, only stretches it out. However, this also means that more of the glacier area will be in the ablation zone, and the glacier will begin to loose mass.
    </details>
</div>

## References

¹ Hester Jiskoot: Glacier Surging. In: Vijay P. Singh, Pratap Singh, Umesh K. Haritashya (Hrsg.): Encyclopedia of Snow, Ice and Glaciers. Springer, Dordrecht 2011, S. 415–428, ISBN 978-90-481-2641-5

## What's next?

[Back to the table of contents](../welcome.ipynb)